Fp Growth algorithm

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
transaction_train = pd.read_csv('/content/gdrive/MyDrive/TRAIN-ARULES.csv', header=None)
transaction_train.head()

,0,1,2
0,order_id,user_id,product_name
1,1483,90,Organic Pink Lemonade Bunny Fruit Snacks
2,1483,90,Dark Chocolate Minis
3,1483,90,"Sparkling Water, Natural Mango Essenced"
4,1483,90,Peach-Pear Sparkling Water


In [6]:
column_names = ["transaction_id", "items"]
dataset = pd.DataFrame()

In [4]:
transaction_id = transaction_train[0].unique()

In [7]:
for tran_id in transaction_id:
  itemslist = ""
  for index, row in transaction_train.iterrows():
    if tran_id == transaction_train[0][index]:
        if not itemslist:
          itemslist = transaction_train[2][index]
        else:
          itemslist += ","+transaction_train[2][index]
  df2 = pd.DataFrame({'transaction_id': [tran_id],
                    'items' : [itemslist]})
  dataset = pd.concat([dataset, df2], ignore_index = True, axis = 0)

dataset.head()

,transaction_id,items
0,order_id,product_name
1,1483,"Organic Pink Lemonade Bunny Fruit Snacks,Dark ..."
2,4595,"Creme De Menthe Thins,Milk Chocolate English T..."
3,7099,"Revive Zero Vitamin Water,VitaminWater Zero™ X..."
4,8382,"Mountain Spring Water,Clementines, Bag,Pineapp..."


In [40]:
dataset_items = dataset.copy()

In [41]:
dataset_items = dataset_items.drop(columns=['transaction_id'])

In [42]:
dataset_items = dataset_items.drop(0)

In [43]:
data_items_table = dataset_items.copy()

In [16]:
dataset_items['items'] = [x.split(',') for x in dataset_items['items']] #dataset_items['items'].split(',')
dataset_items.head()

,items
1,"[Organic Pink Lemonade Bunny Fruit Snacks, Dar..."
2,"[Creme De Menthe Thins, Milk Chocolate English..."
3,"[Revive Zero Vitamin Water, VitaminWater Zero™..."
4,"[Mountain Spring Water, Clementines, Bag, Pin..."
5,"[Truffle, Sea Salt, Black Truffle, 1500 Pale ..."


In [20]:
data_items_list = list()
for idx, row in dataset_items.iterrows():
  data_items_list.append(dataset_items['items'][idx])
print(len(data_items_list))

1418


Encoding transaction into columns

In [21]:
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(data_items_list).transform(data_items_list)
df = pd.DataFrame(te_ary, columns=te.columns_)
df.columns

Index([' & Baby Wipes', ' & orange', ' 10-1/16 Inch', ' 2-Ply Paper Towels',
       ' 3-Ply', ' 50% Reduced Sodium', ' 97% Fat Free', ' 98% Fat Free',
       ' Aged Parmesan and Romano Cheeses Three Cheese Tortellini',
       ' Aged White Cheddar',
       ...
       'Zero Calorie Cola Soda', 'Zero Calorie Lemon Lime Soda',
       'Zero Calorie Tonic Water', 'Zero Go-Go Mixed Berry Vitamin Water',
       'Zero Soda', 'Zero Vitamin Water',
       'Zero XXX Acai Blueberry Pomegranate', 'Zucchini Noodles',
       'Zucchini Squash', 'smartwater® Electrolyte Enhanced Water'],
      dtype='object', length=3685)

In [22]:
df.head()

,& Baby Wipes,& orange,10-1/16 Inch,2-Ply Paper Towels,3-Ply,50% Reduced Sodium,97% Fat Free,98% Fat Free,Aged Parmesan and Romano Cheeses Three Cheese Tortellini,Aged White Cheddar,Aloe,Americano,Antiplaque,Apple Cider,Arame (Sea Vegetable) & Ginger,Baby Bok Choy,Bag,Bags,Bibb) Lettuce,Black Bean & Sweet Corn,Blood Orange,Blueberry on the Bottom Yogurt,Bologna Style,Brew Free! Or Die IPA,Broccoli,Brown Rice,Bunch,Butter,Caffeine Free,California,Calorie Free,Carrots & Kale,Cheddar,Cheddar Cheese,Cherry,Chipotle,Citrus,Clasico,Classic,Coarse,...,Women's MultiVites Gummy Vitamins,World Cuisine Certified Halal Chicken Tikka Masala With Basmati Rice,XL Emerald White Seedless Grapes,XL Pick-A-Size Paper Towel Rolls,Yellow Bell Pepper,Yellow Corn Meal,Yellow Corn Taco Shells,Yellow Corn Tortilla Chips,Yellow Corn Tortillas,Yellow Enriched & Degerminated Corn Meal,Yellow Grape Tomatoes,Yellow Onions,Yellow Potato,Yellow Straightneck Squash,YoBaby Blueberry Apple Yogurt,YoKids Blueberry & Strawberry/Vanilla Yogurt,YoKids Squeeze! Organic Strawberry Flavor Yogurt,YoKids Squeezers Organic Low-Fat Yogurt,YoKids Strawberry Banana/Strawberry Yogurt,Yobaby Organic Plain Yogurt,Yoghurt Blueberry,Yogurt,Yogurt Pretzels,Yotoddler Organic Pear Spinach Mango Yogurt,Yukon Gold Potatoes 5lb Bag,Z Bar Protein Peanut Butter Chocolate Protein Snack Bar,ZBar Organic Chocolate Brownie Energy Snack,ZBar Protein Chocolate Mint Protein Bar,Zen Tea,Zero Calorie Cola,Zero Calorie Cola Soda,Zero Calorie Lemon Lime Soda,Zero Calorie Tonic Water,Zero Go-Go Mixed Berry Vitamin Water,Zero Soda,Zero Vitamin Water,Zero XXX Acai Blueberry Pomegranate,Zucchini Noodles,Zucchini Squash,smartwater® Electrolyte Enhanced Water
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fals

Installing fpgrowth

In [26]:
pip install pyfpgrowth

     |████████████████████████████████| 1.6 MB 31.6 MB/s 
  Created wheel for pyfpgrowth: filename=pyfpgrowth-1.0-py2.py3-none-any.whl size=5504 sha256=105c578a33a24c086a696b01475de6dbaada3eb3aeb60725b749193709d6b274
  Stored in directory: /root/.cache/pip/wheels/73/97/4b/f12ac994f6bbb99597396255435824c73ad3916be1e678be55
Successfully built pyfpgrowth


In [27]:
import pyfpgrowth

patterns = pyfpgrowth. find_frequent_patterns(data_items_list, 10)

In [28]:
rules = pyfpgrowth. generate_association_rules(patterns,0.8)

All the rules generated using fp growth technique

In [29]:
print(rules)

{(' Italian (Flat)',): ((' New England Grown', 'Parsley'), 1.1), (' Italian (Flat)', ' New England Grown'): (('Parsley',), 1.1), (' Italian (Flat)', 'Parsley'): ((' New England Grown',), 1.0), (' New England Grown', 'Parsley'): ((' Italian (Flat)',), 1.0), ('Dairy Free Coconut Milk Yogurt Alternative',): (('Bag of Organic Bananas', 'Medium Navel Orange'), 1.0), ('Medium Navel Orange',): (('Bag of Organic Bananas',), 0.9166666666666666), ('Bag of Organic Bananas', 'Dairy Free Coconut Milk Yogurt Alternative'): (('Medium Navel Orange',), 1.0), ('Bag of Organic Bananas', 'Medium Navel Orange'): (('Dairy Free Coconut Milk Blueberry Yogurt Alternative',), 0.9090909090909091), ('Dairy Free Coconut Milk Yogurt Alternative', 'Medium Navel Orange'): (('Bag of Organic Bananas',), 1.0), ('Dairy Free Coconut Milk Blueberry Yogurt Alternative',): (('Bag of Organic Bananas', 'Medium Navel Orange'), 1.0), ('Bag of Organic Bananas', 'Dairy Free Coconut Milk Blueberry Yogurt Alternative'): (('Medium Na